In [77]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [78]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [79]:
%run -i ../../zkstats/core.py

In [80]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [81]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = (json.loads(open(data_path, "r").read())['col_name'])
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))
# Note, here we make
gt5_data_tensor = data_tensor[data_tensor > 5].reshape(1,-1,1)

create_dummy(data_path, dummy_data_path)
dummy_data = np.array(json.loads(open(dummy_data_path, "r").read())['col_name'])

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))
gt5_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 5].reshape(1,-1,1)

dummy_theory_output = torch.sqrt(torch.var(gt5_dummy_data_tensor, correction = 1))
dummy_data_mean = torch.mean(gt5_dummy_data_tensor)
print(dummy_theory_output)
print(dummy_data_mean)
theory_output = torch.sqrt(torch.var(gt5_data_tensor, correction = 1))
data_mean = torch.mean(gt5_data_tensor)
print(theory_output)
print(data_mean)

tensor(7.1972, dtype=torch.float64)
tensor(17.8173, dtype=torch.float64)
tensor(14.6303)
tensor(49.9367)


In [ ]:
scales = [2]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [82]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        filter = (X>5)
        num_fil_X = torch.sum(filter.double())
        print(num_fil_X)
        fil_mean_X = torch.where(filter, X, 0.0)
        x_mean_cons = torch.abs(torch.sum(fil_mean_X)-num_fil_X*(self.data_mean))<=torch.abs(0.01*num_fil_X*self.data_mean)
        print(x_mean_cons)
        fil_std_X = torch.where(filter, X, self.data_mean)
        print("bool: ",torch.logical_and(torch.abs(torch.sum((fil_std_X-self.data_mean)*(fil_std_X-self.data_mean))-self.w*self.w*(num_fil_X-1))<=torch.abs(0.02*self.w*self.w*(num_fil_X-1)),x_mean_cons) )
        return (torch.logical_and(torch.abs(torch.sum((fil_std_X-self.data_mean)*(fil_std_X-self.data_mean))-self.w*self.w*(num_fil_X-1))<=torch.abs(0.02*self.w*self.w*(num_fil_X-1)),x_mean_cons),self.w)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

tensor(266., dtype=torch.float64)
tensor(True)
bool:  tensor(True)


In [83]:
# Prover/ data owner side
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)

    def forward(self,X):
        filter = (X>5)
        num_fil_X = torch.sum(filter.double())
        print(num_fil_X)
        fil_mean_X = torch.where(filter, X, 0.0)
        x_mean_cons = torch.abs(torch.sum(fil_mean_X)-num_fil_X*(self.data_mean))<=torch.abs(0.01*num_fil_X*self.data_mean)
        fil_std_X = torch.where(filter, X, self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((fil_std_X-self.data_mean)*(fil_std_X-self.data_mean))-self.w*self.w*(num_fil_X-1))<=torch.abs(0.02*self.w*self.w*(num_fil_X-1)),x_mean_cons),self.w)


prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

theory output:  tensor(14.6303)
tensor(300., dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-153448,40664],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":5734,"total_const_size":612,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [84]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 25.950548887252808 seconds
Theory output:  tensor(14.6303)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 14.75
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[10537101196673941533, 17227541574925932677, 11187715152301828262, 1869164017182098189], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4944024074473963208, 53172020031966965, 6785527786944351863, 414441106336697639]]], 'proof': '176ea8eeff21f3aac67cddf3af7cb53a640bed68b132f246ecc1c63e87582df217226576f1f58836639e445dabda705702fd7c32efd81348afe6419f85e476ce1fbe3090ceb3783574a75392dc0af2a2cbc55cc850cda0ee3248bd497596154b08c79289f3f72c8755b603e085d2bd3092af423517ddf341bcace06509b5254b0eb2d53ca692fdb43c785b781a7ee000bf7561be98c9333204b926b710b92f251cc2bdec6303022a2a8eacc6d62a32d4551bb6b40245f47cbbb5ed16f694fe8421c292b3e2ed012912588096163e836afc9265a7d0956d75c0d47bcaad70143707b478c8ef7cde993b8bbe00c30694482ea7facde053493de2477631bf8a5fdd251dcdc81a8ad98ee27fd373a9822cd06968bd340b12ebd2860c48f32f4af7f516df7d24179537ef08eaf571da7beff8ab8c0aeb070bd48da9726ab70df8de4b06a59b2af4863f145bb274827a3c4df43ad20874c97013659e9a6a215ce59b271

In [85]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

num_inputs:  1
prf instances:  [[[10537101196673941533, 17227541574925932677, 11187715152301828262, 1869164017182098189], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4944024074473963208, 53172020031966965, 6785527786944351863, 414441106336697639]]]
proof boolean:  1.0
proof result 1 : 14.75
verified
